# 5-points metrics

### Functions

In [12]:
## "Import" the main structures needed
%run ./final_mainstructures.ipynb

__metaclass__ = type  # Or problems with using super() in Python 2

"""Specific functions for the computations for 5-points metrics"""
       
    
################################### Type I ###################################

class GenericMetricTypeI:
    """A generic 5-point metric of type I"""  
    
    def __init__(self, xi, beta):
        self.type = 'I'
        self.X = FiniteSet(Set(['x1', 'x2', 'x3', 'x4', 'x5']))    
        # Construct split system with parameters xi, beta
        self.S = SplitSystem(self.X)
        self.S.splits.extend([
        Split(self.X, Set(['x1']), xi[0]),
        Split(self.X, Set(['x2']), xi[1]),
        Split(self.X, Set(['x3']), xi[2]),
        Split(self.X, Set(['x4']), xi[3]),
        Split(self.X, Set(['x5']), xi[4]),
        Split(self.X, Set(['x1', 'x2']), beta[0]),
        Split(self.X, Set(['x2', 'x3']), beta[1]),
        Split(self.X, Set(['x3', 'x4']), beta[2]),
        Split(self.X, Set(['x4', 'x5']), beta[3]),
        Split(self.X, Set(['x5', 'x1']), beta[4]) ])
        # Construct metric, here simply given by the split system
        self.d = self.S.d_alpha
        self.xi = xi
        self.beta = beta
        # Compute and store the distances for calculations of polytopes
        # Use it to check if it yields in fact a metric
        self.distances, self.is_metric = compute_distances_5pts(self.X, self.d)
        self.d12, self.d13, self.d14, self.d15, self.d23 = self.distances[:5]
        self.d24, self.d25, self.d34, self.d35, self.d45 = self.distances[5:]
    
    def __repr__(self):
        write =  "A generic 5-point metric of type " + self.type + ", with parameters:\n"
        write += "Xi = " + str(self.xi) + ", Beta = " + str(self.beta) + "\n"
        return write
        

################################### Type II and III ###################################

def d_prime(a,b):
    """K_{2,3} metric, to use for type II and III"""
    if a==b:
        return 0
    elif (a == 'z1' and b =='z2') or (b == 'z1' and a =='z2'):
        return 2
    elif (a == 'y1' and b =='y2') or (b == 'y1' and a =='y2'):
        return 2
    elif (a == 'y1' and b =='y3') or (b == 'y1' and a =='y3'):
        return 2
    elif (a == 'y2' and b =='y3') or (b == 'y2' and a =='y3'):
        return 2
    elif (a == 'z1' or a =='z2') and (b == 'y1' or b == 'y2' or b == 'y3'):
        return 1
    elif (b == 'z1' or b =='z2') and (a == 'y1' or a == 'y2' or a == 'y3'):
        return 1
    
def construct_metric(S, alpha):
    """Return d(a,b) for d = split decomposable part + alpha*prime part
    where S = type_II_split_system(zeta, eta, beta) for the split system."""
    def metric(a,b):
        distance = S.d_alpha(a,b) + alpha*d_prime(a,b)
        return distance
    return metric

    
class TypeIIorIII():
    """Since type II and III share a lot, we make a superclass"""  
    
    def __init__(self, metric_type, zeta, eta, beta, alpha):
        self.type = metric_type
        # Use the same labels as in the article
        self.X = FiniteSet(Set(['z1', 'z2', 'y1', 'y2', 'y3']))   
        # Start constructing split system with parameters (only last split will be different)
        self.S = SplitSystem(self.X)
        self.S.splits.extend([
        Split(self.X, Set(['z1']), zeta[0]),
        Split(self.X, Set(['z2']), zeta[1]),
        Split(self.X, Set(['y1']), eta[0]),
        Split(self.X, Set(['y2']), eta[1]),
        Split(self.X, Set(['y3']), eta[2]),   
        Split(self.X, Set(['y1', 'z1']), beta[0]),
        Split(self.X, Set(['z1', 'y2']), beta[1]),
        Split(self.X, Set(['y2', 'z2']), beta[2]) ])
        # Save parameters
        self.zeta = zeta
        self.eta = eta
        self.beta = beta
        self.alpha = alpha

    def __repr__(self):
        write =  "A generic 5-point metric of type " + self.type + ", with parameters:\n"
        write += "Zeta = " + str(self.zeta) + ", Eta = " + str(self.eta)
        write += ", Beta = " + str(self.beta) + ", Alpha = " + str(self.alpha) + "\n"
        return write
        
    
class GenericMetricTypeII(TypeIIorIII):
    """A generic 5-point metric of type II"""  
    
    def __init__(self, zeta, eta, beta, alpha):
        super(GenericMetricTypeII, self).__init__('II', zeta, eta, beta, alpha)
        # Add the missing split that is specific for type II
        self.S.splits.extend([ Split(self.X, Set(['z2', 'y1']), beta[3]) ])
        # Construct metric
        self.d = construct_metric(self.S, alpha)
        # Compute and store the distances for calculations of polytopes
        # Use it to check if it yields in fact a metric
        self.distances, self.is_metric = compute_distances_5pts(self.X, self.d)
        self.d12, self.d13, self.d14, self.d15, self.d23 = self.distances[:5]
        self.d24, self.d25, self.d34, self.d35, self.d45 = self.distances[5:]
       
        
class GenericMetricTypeIII(TypeIIorIII):
    """A generic 5-point metric of type III"""  
    
    def __init__(self, zeta, eta, beta, alpha):
        super(GenericMetricTypeIII, self).__init__('III', zeta, eta, beta, alpha)
        # Add the missing split that is specific for type III
        self.S.splits.extend([ Split(self.X, Set(['z2', 'y3']), beta[3]) ])
        # Construct metric
        self.d = construct_metric(self.S, alpha)
        # Compute and store the distances for calculations of polytopes
        # Use it to check if it yields in fact a metric
        self.distances, self.is_metric = compute_distances_5pts(self.X, self.d)
        self.d12, self.d13, self.d14, self.d15, self.d23 = self.distances[:5]
        self.d24, self.d25, self.d34, self.d35, self.d45 = self.distances[5:]
        
################################### Lipschitz and fundamental polytopes ###################################

def compute_distances_5pts(X, d):
    """Compute distances needed for Lipschitz and fundamental polytope calculations
    Called in initialiser of the classes to avoid computing twice.
    Also check whether any distance is zero, then is only a pseudometric."""
    points = sorted(X.set)
    d12 = d(points[0], points[1])
    d13 = d(points[0], points[2])
    d14 = d(points[0], points[3])
    d15 = d(points[0], points[4])
    d23 = d(points[1], points[2])
    d24 = d(points[1], points[3])   
    d25 = d(points[1], points[4])
    d34 = d(points[2], points[3])
    d35 = d(points[2], points[4])
    d45 = d(points[3], points[4])
    distances = (d12, d13, d14, d15, d23, d24, d25, d34, d35, d45)
    if 0 in distances:
        is_metric = False
    else:
        is_metric = True
    return (distances, is_metric)


def lipschitz_polytope_5pointsset(metric):
    """Constructs the Lipschitz polytope for 5 points sets metric on X depending on type.
    Provide an instance of one of the GenericMetricType classes."""
    equation = [[0,1,1,1,1,1]]
    inequations = ([metric.d12,1,-1,0,0,0], [metric.d12,-1,1,0,0,0], [metric.d13,1,0,-1,0,0], [metric.d13,-1,0,1,0,0], 
                   [metric.d14,1,0,0,-1,0], [metric.d14,-1,0,0,1,0], [metric.d15,1,0,0,0,-1], [metric.d15,-1,0,0,0,1], 
                   [metric.d23,0,1,-1,0,0], [metric.d23,0,-1,1,0,0], [metric.d24,0,1,0,-1,0], [metric.d24,0,-1,0,1,0], 
                   [metric.d25,0,1,0,0,-1], [metric.d25,0,-1,0,0,1], [metric.d34,0,0,1,-1,0], [metric.d34,0,0,-1,1,0], 
                   [metric.d35,0,0,1,0,-1], [metric.d35,0,0,-1,0,1], [metric.d45,0,0,0,1,-1], [metric.d45,0,0,0,-1,1])
    return Polyhedron(eqns=equation, ieqs=inequations)

      # intersect with X.RX ?

def fundamental_polytope_5pointsset(metric):
    """Constructs the Fundamental polytope for 5 points sets metric on X depending on type.
    Provide an instance of one of the GenericMetricType classes."""
    I = metric.X.Ivectors
    points = sorted(metric.X.set)
    e12 = (I[points[0]] - I[points[1]]) / metric.d12
    e13 = (I[points[0]] - I[points[2]]) / metric.d13
    e14 = (I[points[0]] - I[points[3]]) / metric.d14
    e15 = (I[points[0]] - I[points[4]]) / metric.d15
    e23 = (I[points[1]] - I[points[2]]) / metric.d23
    e24 = (I[points[1]] - I[points[3]]) / metric.d24
    e25 = (I[points[1]] - I[points[4]]) / metric.d25
    e34 = (I[points[2]] - I[points[3]]) / metric.d34
    e35 = (I[points[2]] - I[points[4]]) / metric.d35
    e45 = (I[points[3]] - I[points[4]]) / metric.d45
    e21 = -e12
    e31 = -e13
    e41 = -e14
    e51 = -e15
    e32 = -e23
    e42 = -e24
    e52 = -e25
    e43 = -e34
    e53 = -e35
    e54 = -e45
    return Polyhedron(vertices=[e12, e13, e14, e15, e23, e24, e25, e34, e35, e45,
                                e21, e31, e41, e51, e32, e42, e52, e43, e53, e54])



# A enlever en python 3 : metaclass en haut, vider les super()

In [9]:
testZ = Polyhedron(base_ring = ZZ, ambient_dim = 5)
testZ.ehrhart_polynomial()

0

In [102]:
xi = [1,1,1,1,0]
beta = [0,0,1,0,0]
M = GenericMetricTypeI(xi, beta)
M.S.is_metric()

False

In [24]:
zeta = [1,0]
eta = [0,1,0]
beta = [0,1,0,1]
alpha = 1
M = GenericMetricTypeII(zeta, eta, beta, alpha)
#results(M)
M

A generic 5-point metric of type II, with parameters:
Zeta = [1, 0], Eta = [0, 1, 0], Beta = [0, 1, 0, 1], Alpha = 1


# Custom parameters

#### Type I

In [13]:
# Choose parameters
xi = [1,1,0,1,0]
beta = [1,1,0,0,0]

# Compute
M = GenericMetricTypeI(xi,beta)
rows = [results(M)]
filename = "5pts_" + M.type + "_xi" + ''.join(map(str, xi)) + "_beta" + ''.join(map(str, beta))

## Write output in files, one .txt and one .tex
T = table(rows, header_row = header, frame = True).transpose()
f = open(filename + ".txt", 'w')
f.write(str(T))
f.close()
flatex = open(filename + ".tex", 'w')
flatex.write(str(latex(T)))
flatex.close()

##### Type II or III

In [13]:
# Choose parameters
wanted_type = "II"
zeta = [0,0]
eta = [0,0,0]
beta = [0,0,0,0]
alpha = 1

# Compute
if wanted_type == "II":
    M = GenericMetricTypeII(zeta, eta, beta, alpha)
elif wanted_type == "III":
    M = GenericMetricTypeIII(zeta, eta, beta, alpha)

rows = [results(M)]
filename = "5pts_" + M.type + "_zeta" + ''.join(map(str, zeta)) + "_eta" + ''.join(map(str, eta)) 
filename += "_beta" + ''.join(map(str, beta)) + "_alpha" + str(alpha)


# Write output in files, one .txt and one .tex
T = table(rows, header_row = header, frame = True).transpose()
f = open(filename + ".txt", 'w')
f.write(str(T))
f.close()
flatex = open(filename + ".tex", 'w')
flatex.write(str(latex(T)))
flatex.close()

# Common

In [14]:
def countings(typ):
    # List all combination possibilities for parameters taking value 0 or 1
    parameter_possibilities = IntegerVectors(None, 10, min_part=0, max_part=1).list()
    parameter_possibilities.reverse()
    
    # Special for periphery parameters zero
    #parameter_possibilities = IntegerVectors(None, 5, min_part=0, max_part=1).list()
    #for i in range(0, len(parameter_possibilities)):
    #    parameter_possibilities[i] = [0,0,0,0,0] + list(parameter_possibilities[i])
    #parameter_possibilities.reverse()

    # List all metrics yielded by combinations of parameters of value 0 or 1
    metrics_01 = [] 
    # Of the previous, list one metric for each occuring f-vector of Fundamental polytope
    metrics_01_fvectors_Fund = [] 
    occuring_fvectors_Fund = []
    # Dictionnary for later comparison
    dic_metricparam_fvectF = {}
    # List of parameter combinations not yielding metrics
    not_metrics_01 = []
    
    for p in parameter_possibilities:
        # Define parameters and compute according to type of generic metric
        if typ == "I":
            xi = p[:5]
            beta = p[5:]
            M = GenericMetricTypeI(xi, beta)
        elif typ == "II" or typ == "III":
            zeta = p[:2]
            eta = p[2:5]
            beta = p[5:9]
            alpha = p[9]
            if typ == "II":
                M = GenericMetricTypeII(zeta, eta, beta, alpha)
            elif typ == "III":
                M = GenericMetricTypeIII(zeta, eta, beta, alpha)
                
        # If it yields a metric, compute f-vector of fundamental polytope       
        if M.is_metric:
            metrics_01.append(p)
            F = fundamental_polytope_5pointsset(M)
            f = F.f_vector()
            #f = '.'.join(map(str, f))
            dic_metricparam_fvectF[p] = f
            if not f in occuring_fvectors_Fund:
                occuring_fvectors_Fund.append(f)
                metrics_01_fvectors_Fund.append(p)
        else:
            not_metrics_01.append(p)
            
    print(len(metrics_01), len(metrics_01_fvectors_Fund), len(not_metrics_01))
    return (metrics_01, metrics_01_fvectors_Fund, occuring_fvectors_Fund, 
            dic_metricparam_fvectF, 
            not_metrics_01)
        
## For every type, we will use this function to obtain the following values, adding the type as subscript (f.e. M_II)
# M = list of parameter combinations of value 0 or 1 that yield metrics
# Mf = subset of M_I; taking only one metric per occuring f-vector of the Fundamental polytope
# fvs = list of the occuring f-vectors of Fundamental polytope for the M_I yielded metrics
# dic = dictionnary pairing the parameter combinations yielding metrics, and the f-vector of Fund. polytope
# NM = list of parameter combinations of value 0 or 1 that do not yield metrics   

M_I,   Mf_I,   fvs_I,   dic_I,   NM_I   = countings("I")
M_II,  Mf_II,  fvs_II,  dic_II,  NM_II  = countings("II")
M_III, Mf_III, fvs_III, dic_III, NM_III = countings("III")


(748, 55, 276)
(858, 80, 166)
(836, 86, 188)


In [20]:
from collections import Counter
c = Counter()
fvs=fvs_I
for xs in fvs:
     for x in set(xs):
         c[x] += 1
c

Counter({1: 55, 14: 14, 16: 13, 20: 11, 10: 10, 42: 9, 12: 9, 18: 8, 36: 7, 48: 7, 28: 6, 30: 6, 32: 6, 40: 6, 44: 6, 50: 6, 54: 6, 60: 6, 24: 5, 34: 5, 52: 5, 58: 5, 66: 5, 72: 5, 62: 4, 38: 3, 64: 3, 70: 3, 76: 3, 84: 3, 26: 2, 46: 2, 56: 2, 74: 2, 78: 2, 80: 2, 86: 2, 88: 2, 90: 2, 100: 2, 102: 2, 8: 1, 140: 1, 22: 1, 68: 1, 82: 1, 92: 1, 94: 1, 104: 1, 106: 1, 114: 1, 116: 1, 120: 1, 122: 1})

In [18]:
from collections import Counter
fvs=fvs_I
for i in range(len(fvs)):
    a=fvs[i].list()
    fvs[i]=join
    
Counter(fvs)

AttributeError: 'list' object has no attribute 'list'

In [5]:
doublons = {}
for k in dic_I:
    if 'sign' in k:
        doublons[k['sign']] = doublons.get(k['sign'], 0) + 1
doublons

{}

In [21]:
def periphzero_countings(typ):
    parameter_periphzero = IntegerVectors(None, 5, min_part=0, max_part=1).list()
    for i in range(0, len(parameter_periphzero)):
        parameter_periphzero[i] = [0,0,0,0,0] + list(parameter_periphzero[i])
    parameter_periphzero.reverse()

    # List all metrics yielded by combinations of parameters of value 0 or 1
    metrics_01 = [] 
    # Of the previous, list one metric for each occuring f-vector of Fundamental polytope
    metrics_01_fvectors_Fund = [] 
    occuring_fvectors_Fund = []
    # Dictionnary for later comparison
    dic_metricparam_fvectF = {}
    # List of parameter combinations not yielding metrics
    not_metrics_01 = []
    
    for p in parameter_periphzero:
        # Define parameters and compute according to type of generic metric
        if typ == "I":
            xi = p[:5]
            beta = p[5:]
            M = GenericMetricTypeI(xi, beta)
        elif typ == "II" or typ == "III":
            zeta = p[:2]
            eta = p[2:5]
            beta = p[5:9]
            alpha = p[9]
            if typ == "II":
                M = GenericMetricTypeII(zeta, eta, beta, alpha)
            elif typ == "III":
                M = GenericMetricTypeIII(zeta, eta, beta, alpha)
                
        # If it yields a metric, compute f-vector of fundamental polytope       
        if M.is_metric:
            metrics_01.append(p)
            F = fundamental_polytope_5pointsset(M)
            f = F.f_vector()
            #f = '.'.join(map(str, f))
            #dic_metricparam_fvectF[p] = f
            if not f in occuring_fvectors_Fund:
                occuring_fvectors_Fund.append(f)
                metrics_01_fvectors_Fund.append(p)
        else:
            not_metrics_01.append(p)
            
    print(len(metrics_01), len(metrics_01_fvectors_Fund), len(not_metrics_01))
    return (metrics_01, metrics_01_fvectors_Fund, occuring_fvectors_Fund, 
            #dic_metricparam_fvectF, 
            not_metrics_01)


# Comment out the right parameter_possibilities, and dic_ twice, change title in latex files
pM_I,   pMf_I,   pfvs_I,    pNM_I   = periphzero_countings("I")
pM_II,  pMf_II,  pfvs_II,   pNM_II  = periphzero_countings("II")
pM_III, pMf_III, pfvs_III,  pNM_III = periphzero_countings("III")    

(11, 3, 21)
(21, 9, 11)
(19, 9, 13)


In [9]:
for x in dic_II:
    if dic_II[x]==dic_II[Mf_II[0]]:
        print(x,dic_II[x])

([1, 1, 1, 1, 1, 0, 1, 0, 1, 0], (1, 20, 80, 114, 54, 1))
([1, 1, 1, 1, 1, 1, 0, 0, 1, 1], (1, 20, 80, 114, 54, 1))
([1, 1, 1, 1, 1, 1, 1, 1, 1, 0], (1, 20, 80, 114, 54, 1))
([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], (1, 20, 80, 114, 54, 1))
([1, 1, 1, 1, 1, 0, 1, 1, 0, 1], (1, 20, 80, 114, 54, 1))
([1, 1, 1, 1, 1, 1, 0, 1, 0, 0], (1, 20, 80, 114, 54, 1))


# Latex files generation

In [18]:
# Header to appear in latex tables
header_latex = ["", "Parameters", "Splits", "Compatible", "\\lip=\\zon", 
          "\\fv\\fun", "\\ip\\fun", "\\fv\\lip", "\\ip\\lip", "\\fv\\zon", "\\ip\\zon",
          "\\ehr\\zons", "\\ip\\zons"] * 2


def extract_parameters(typ, parameters):
    if typ == "I":
        xi = parameters[:5]
        beta = parameters[5:]
        return (xi, beta)     
    elif typ == "II" or typ == "III":
        zeta = parameters[:2]
        eta = parameters[2:5]
        beta = parameters[5:9]
        alpha = parameters[9]
        return (zeta, eta, beta, alpha)

def get_occurences_number(typ, parameters):
    if typ == "I":
        return sum(1 for x in dic_I.values() if x==dic_I[parameters])
    elif typ == "II":
        return sum(1 for x in dic_II.values() if x==dic_II[parameters])        
    elif typ == "III":    
        return sum(1 for x in dic_III.values() if x==dic_III[parameters])    


def subtable(typ, parameters, case_number, occurences):
    if typ == "I":
        xi, beta = extract_parameters(typ, parameters)
        param = ''.join(map(str, xi)) + " " + ''.join(map(str, beta))
        r = results(GenericMetricTypeI(xi,beta))
        r[0] = "\\parone " + param
        tikzmacro = "\\fiveone " + param    
    elif typ == "II" or typ == "III":
        zeta, eta, beta, alpha = extract_parameters(typ, parameters)
        param = ''.join(map(str, zeta)) + ''.join(map(str, eta)) + " " + ''.join(map(str, beta))  + str(alpha)
        if typ =="II":
            r = results(GenericMetricTypeII(zeta, eta, beta, alpha))
            r[0] = "\\partwothree " + param
            tikzmacro = "\\fivetwo " + param
        elif typ == "III":
            r = results(GenericMetricTypeIII(zeta, eta, beta, alpha))
            r[0] = "\\partwothree " + param
            tikzmacro = "\\fivethree " + param
    first_row = "\\drawwithnumber{{\\case{{{}}}{{{}}}}}{{{}}}".format(case_number, occurences, tikzmacro)
    r.insert(0, first_row)
    return r           

def generate_latex_files(typ, cases_list, addtofilename=""):
    nb_cases = len(cases_list)
    cases_on_last_page = nb_cases%4
    
    # Take batches of 4 cases, compute the results into a table and save as file
    for a in range(0, (nb_cases//4)*4, 4):
        current_4_cases = cases_list[a:a+4]
        filename = "5pts{}_{}_{}".format(addtofilename, typ, str((a+4)/4).zfill(2))
        rows = []
        rows.append(header_latex)
        rows.append(subtable(typ, current_4_cases[0], a+1, get_occurences_number(typ, current_4_cases[0])))
        rows[-1] = rows[-1] + subtable(typ, current_4_cases[2], a+3, get_occurences_number(typ, current_4_cases[2]))
        rows.append(subtable(typ, current_4_cases[1], a+2, get_occurences_number(typ, current_4_cases[1])))
        rows[-1] = rows[-1] + subtable(typ, current_4_cases[3], a+4, get_occurences_number(typ, current_4_cases[3]))
        T = table(rows, frame = True, align = 'center').transpose()
        flatex = open(filename + ".tex", 'w')
        flatex.write("\\resizebox{!}{\\myheighttables\height}{ \n") # \\begin{landscape} \n
        flatex.write(str(latex(T)))
        flatex.write("\n }") # \\end{landscape} 
        flatex.close()

    # Put the last 1, 2 or 3 results on the last page, with other slots empty
    if cases_on_last_page:
        a = (nb_cases//4)*4
        last_cases = cases_list[a:a+4]
        last_tables = []
        for i in range(0, cases_on_last_page):
            last_tables.append(subtable(typ, last_cases[i], a+i+1, get_occurences_number(typ, last_cases[i])))
        for i in range(0, 4-cases_on_last_page):
            last_tables.append([""]*13)
        filename = "5pts{}_{}_{}".format(addtofilename, typ, str((a+4)/4).zfill(2))
        rows = []
        rows.append(header_latex)
        rows.append(last_tables[0])
        rows[-1] = rows[-1] + last_tables[2]
        rows.append(last_tables[1])
        rows[-1] = rows[-1] + last_tables[3]
        T = table(rows, frame = True, align = 'center').transpose()
        flatex = open(filename + ".tex", 'w')
        flatex.write("\\resizebox{!}{\\myheighttables\height}{ \n") # \\begin{landscape} \n
        flatex.write(str(latex(T)))
        flatex.write("\n }") # \\end{landscape} 
        flatex.close()
         

In [19]:
## Produce latex files with 4 results per page (one file for each page)

generate_latex_files("I",   Mf_I)
generate_latex_files("II",  Mf_II)
generate_latex_files("III", Mf_III)

#generate_latex_files("I",   pM_I,   "periph")
#generate_latex_files("II",  pM_II,  "periph")
#generate_latex_files("III", pM_III, "periph")

## Parameters for each example

In [8]:
def example_row(typ, parameters):
    r = []
    if typ == "I":
        xi, beta = extract_parameters(typ, parameters)
        param = ''.join(map(str, xi)) + " " + ''.join(map(str, beta))
        r.append("\\parone " + param) 
        #r = results(GenericMetricTypeI(xi,beta))
        M = GenericMetricTypeI(xi,beta)   
    elif typ == "II" or typ == "III":
        zeta, eta, beta, alpha = extract_parameters(typ, parameters)
        param = ''.join(map(str, zeta)) + ''.join(map(str, eta)) + " " + ''.join(map(str, beta))  + str(alpha)
        r.append("\\partwothree " + param)
        if typ =="II":
            #r = results(GenericMetricTypeII(zeta, eta, beta, alpha))
            M = GenericMetricTypeII(zeta, eta, beta, alpha)
        elif typ == "III":
            #r = results(GenericMetricTypeIII(zeta, eta, beta, alpha))
            M = GenericMetricTypeIII(zeta, eta, beta, alpha)
    Lip = lipschitz_polytope_5pointsset(M)
    Z = M.S.zonotope()
    gen_vec = M.S.zonotope_generating_vectors()
    
    Zscaled = M.X.n * Z
    gen_vec_scaled = []
    for vec in gen_vec:
        gen_vec_scaled.append(M.X.n * vec)
    h(t) = ehrhart_poly(gen_vec_scaled, M.X)
    intptsscaled = h(1)
    
    r = r + [M.distances,len(Lip.integral_points()), Z.f_vector(), len(Z.integral_points()), intptsscaled]
    return r  

def table_similars(typ,example,ex_number,dic,fvector):
    rows = [["\\Ex{{{}}}".format(ex_number) + "\\quad {{{}}}".format("\\fv\\fun = " + str(fvector)),"","","","",""]]
    rows.append(["Parameters", "Distances", "\\ip\\lip", "\\fv\\zon", "\\ip\\zon", "\\ip\\zons"])
    liste = []
    for param in dic.keys():
        if dic[param]==fvector:
            liste.append(param)
    for similar in liste:
        rows.append(example_row(typ, similar))
    return table(rows, frame = True, align = 'center')

def generate_similars_tables(typ, examples_list, dic):
    filename = "5pts_{}_similars".format(typ)
    flatex = open(filename + ".tex", 'w')
    for i in range(len(examples_list)):
        example = examples_list[i]
        fvector = dic[example]
        table = table_similars(typ, example, i+1, dic, fvector)
        flatex.write(str(latex(table)))
        flatex.write("\n \n")
        print(str(i))
    flatex.close()
    

In [9]:
generate_similars_tables("I",   Mf_I,   dic_I)
generate_similars_tables("II",  Mf_II,  dic_II)
generate_similars_tables("III", Mf_III, dic_III)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85


{


## Comparisons

In [ ]:
def compare(typ, Mf, dic):
    
    for example in Mf:
        same_f_list = []
        for i in dic.keys():
            if dic[i]==dic[example]:
                
                
        
        sum(1 for x in dic_I.values() if x==dic_I[parameters])

In [45]:
set(dic_I.items()) - set(dic_II.items())

TypeError: mutable vectors are unhashable

In [ ]:
from collections import Counter
    
print(len(Counter(dic_I).keys()))
print(Counter(fvs_I).substract(Counter(fvs_II)))
#print(Counter(dic_I).values())
#print(list(sorted(set(Counter(dic_I).values()))))
#print(len(set(dic_I)) )#, list(set(f_vectors)-set(list_f_doubled)))

In [24]:
for pz in pM_III:
    if pz not in Mf_III:
        print(pz)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [ ]:
print(set(pM_I).issubset(Mf_I), pM_II in Mf_II, pM_III in Mf_III)
print(pfvs_I in fvs_I, pfvs_II in fvs_II, pfvs_III in fvs_III)

In [31]:
from collections import Counter

list_f_doubled = []
for i in doubles_of_f:
    list_f_doubled.append(i[0])
    
print(Counter(list_f_doubled).keys())
print(Counter(list_f_doubled).values())
print(list(sorted(set(Counter(list_f_doubled).values()))))
print (len(set(list_f_doubled)), list(set(f_vectors)-set(list_f_doubled)))

['1.16.66.100.50.1', '1.14.50.66.30.1', '1.14.50.64.28.1', '1.18.72.102.48.1', '1.18.66.90.42.1', '1.20.76.102.46.1', '1.16.68.104.52.1', '1.10.34.44.20.1', '1.16.62.88.42.1', '1.12.42.58.28.1', '1.18.74.106.50.1', '1.16.62.84.38.1', '1.20.82.116.54.1', '1.10.32.40.18.1', '1.14.48.64.30.1', '1.16.60.80.36.1', '1.10.34.42.18.1', '1.14.52.72.34.1', '1.14.54.76.36.1', '1.14.48.62.28.1', '1.10.34.48.24.1', '1.12.40.50.22.1', '1.12.48.72.36.1', '1.10.30.36.16.1', '1.16.54.66.28.1', '1.12.42.54.24.1', '1.8.24.32.16.1', '1.20.84.122.58.1', '1.14.48.60.26.1', '1.18.66.86.38.1', '1.16.64.92.44.1', '1.14.44.50.20.1', '1.12.40.48.20.1', '1.14.58.88.44.1', '1.16.62.86.40.1', '1.10.32.42.20.1', '1.16.58.76.34.1', '1.20.80.114.54.1', '1.16.58.78.36.1', '1.14.52.70.32.1', '1.14.56.84.42.1', '1.12.42.56.26.1', '1.14.44.54.24.1', '1.12.40.52.24.1', '1.20.72.94.42.1', '1.14.52.74.36.1', '1.12.44.60.28.1', '1.10.28.30.12.1', '1.10.32.36.14.1', '1.18.78.120.60.1', '1.10.32.38.16.1', '1.18.72.100.46.1']
[4

In [50]:
from collections import Counter

list_f_doubled_II = []
for i in doubles_of_f_II:
    list_f_doubled_II.append(i[0])
    
print(Counter(list_f_doubled_II).keys())
print(Counter(list_f_doubled_II).values())
print(list(sorted(set(Counter(list_f_doubled_II).values()))))
print (len(list_f_doubled_II), len(set(list_f_doubled_II)), list(set(f_vectorsII)-set(list_f_doubled_II)))
print((list(set(f_vectorsII)-set(f_vectors))))
print((list(set(f_vectors)-set(f_vectorsII))))
print((list(set(f_vectorsII).intersection(set(f_vectors)))))

['1.14.50.66.30.1', '1.16.64.90.42.1', '1.20.80.112.52.1', '1.20.80.114.54.1', '1.16.60.82.38.1', '1.18.68.90.40.1', '1.20.76.102.46.1', '1.20.72.94.42.1', '1.18.74.106.50.1', '1.12.42.58.28.1', '1.14.52.68.30.1', '1.16.62.84.38.1', '1.16.64.92.44.1', '1.20.82.116.54.1', '1.20.86.128.62.1', '1.18.76.112.54.1', '1.14.54.76.36.1', '1.10.32.40.18.1', '1.16.58.72.30.1', '1.18.70.98.46.1', '1.16.60.80.36.1', '1.14.50.68.32.1', '1.18.74.104.48.1', '1.16.56.70.30.1', '1.16.62.82.36.1', '1.18.68.92.42.1', '1.14.48.62.28.1', '1.10.34.48.24.1', '1.14.54.78.38.1', '1.12.40.50.22.1', '1.12.48.72.36.1', '1.10.30.36.16.1', '1.18.62.78.34.1', '1.16.54.66.28.1', '1.12.42.54.24.1', '1.8.24.32.16.1', '1.20.84.122.58.1', '1.14.48.60.26.1', '1.18.66.86.38.1', '1.12.38.46.20.1', '1.16.64.94.46.1', '1.14.44.50.20.1', '1.12.40.48.20.1', '1.14.50.62.26.1', '1.16.62.86.40.1', '1.10.32.42.20.1', '1.16.58.76.34.1', '1.16.58.78.36.1', '1.18.68.88.38.1', '1.14.52.72.34.1', '1.14.52.70.32.1', '1.14.48.64.30.1', '1.

In [5]:
from collections import Counter

list_f_doubled_III = []
for i in doubles_of_f_III:
    list_f_doubled_III.append(i[0])
    
print(Counter(list_f_doubled_III).keys())
print(Counter(list_f_doubled_III).values())
print(list(sorted(set(Counter(list_f_doubled_III).values()))))
print (len(list_f_doubled_III), len(set(list_f_doubled_III)), list(set(f_vectorsIII)-set(list_f_doubled_III)))
print("\n")
print((list(set(f_vectorsIII)-set(f_vectors))))
print("\n")
print((list(set(f_vectorsIII)-set(f_vectorsII))))
print("\n")
print((list(set(f_vectors)-set(f_vectorsII))))
print("\n")
print((list(set(f_vectorsIII).intersection(set(f_vectors)))))
print("\n")
print((list(set(f_vectorsIII).intersection(set(f_vectorsII)))))

['1.14.50.66.30.1', '1.16.62.90.44.1', '1.18.72.102.48.1', '1.16.64.90.42.1', '1.20.80.112.52.1', '1.20.80.114.54.1', '1.18.70.100.48.1', '1.20.84.124.60.1', '1.18.68.90.40.1', '1.20.76.102.46.1', '1.20.72.94.42.1', '1.18.74.106.50.1', '1.12.42.58.28.1', '1.14.52.68.30.1', '1.16.62.84.38.1', '1.16.64.92.44.1', '1.20.82.116.54.1', '1.20.86.128.62.1', '1.18.76.112.54.1', '1.10.32.40.18.1', '1.16.58.72.30.1', '1.18.70.98.46.1', '1.16.60.80.36.1', '1.14.50.68.32.1', '1.18.74.104.48.1', '1.14.54.76.36.1', '1.16.62.82.36.1', '1.18.68.92.42.1', '1.14.48.62.28.1', '1.10.34.48.24.1', '1.14.54.78.38.1', '1.12.40.50.22.1', '1.12.48.72.36.1', '1.10.30.36.16.1', '1.18.62.78.34.1', '1.16.54.66.28.1', '1.12.42.54.24.1', '1.8.24.32.16.1', '1.20.84.122.58.1', '1.14.48.60.26.1', '1.18.66.86.38.1', '1.16.60.84.40.1', '1.12.38.46.20.1', '1.16.64.94.46.1', '1.14.44.50.20.1', '1.12.40.48.20.1', '1.14.50.62.26.1', '1.16.62.86.40.1', '1.10.32.42.20.1', '1.16.58.76.34.1', '1.16.58.78.36.1', '1.18.68.88.38.1', 

NameError: name 'f_vectors' is not defined

In [11]:
set(uniqueF_typeImetrics).intersection(uniqueF_typeIImetrics)
#mais non il faut comparer les f vectors

{[0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
 [1, 0, 0, 0, 0, 1, 1, 1, 0, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 0, 0],
 [1, 1, 0, 0, 0, 1, 1, 0, 0, 1],
 [1, 1, 0, 0, 0, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 1, 0, 0, 0, 1],
 [1, 1, 1, 1, 0, 1, 0, 0, 1, 1],
 [1, 1, 1, 1, 0, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 0, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]}

(858, 80, 778)

(836, 86, 750)

10000 & 01100 \\
00000 & 11100 \\
11110 & 00000 \\
11000 & 01100 \\
11000 & 00110 \\
10000 & 11100 \\
10000 & 11001 \\
00000 & 11110 \\
11111 & 00000 \\
11110 & 10000 \\
11110 & 00010 \\
11100 & 10001 \\
11100 & 00110 \\
11010 & 01100 \\
11010 & 00110 \\
11000 & 11100 \\
11000 & 11001 \\
11000 & 10110 \\
11000 & 01110 \\
10100 & 11100 \\
10000 & 11011 \\
00000 & 11111 \\
11111 & 10000 \\
11110 & 11000 \\
11110 & 10100 \\
11110 & 10010 \\
11110 & 10001 \\
11110 & 00011 \\
11100 & 11100 \\
11100 & 10110 \\
11100 & 10011 \\
11100 & 00111 \\
11010 & 11001 \\
11010 & 10110 \\
11010 & 01110 \\
11000 & 11110 \\
11111 & 11000 \\
11111 & 10100 \\
11110 & 11100 \\
11110 & 11001 \\
11110 & 10110 \\
11110 & 10011 \\
11110 & 01011 \\
11100 & 11110 \\
11000 & 11111 \\
10100 & 11111 \\
11111 & 11100 \\
11110 & 11110 \\
11110 & 11011 \\
11110 & 10111 \\
11100 & 11111 \\
11010 & 11111 \\
11111 & 11110 \\
11110 & 11111 \\
11111 & 11111 \\


My name is John, and I am 0 0


0
4
8
12
16
20
24
28
32
36
40
44
48


NameError: name 'X' is not defined

NameError: name 'M' is not defined

[['1.10.28.30.12.1', [1, 0, 0, 0, 0, 0, 0, 1, 1, 0]],
 ['1.10.28.30.12.1', [0, 1, 0, 0, 0, 0, 0, 1, 1, 0]],
 ['1.10.28.30.12.1', [0, 1, 0, 0, 0, 0, 0, 0, 1, 1]],
 ['1.10.28.30.12.1', [0, 0, 1, 0, 0, 1, 0, 0, 0, 1]],
 ['1.10.28.30.12.1', [0, 0, 1, 0, 0, 0, 0, 0, 1, 1]],
 ['1.10.28.30.12.1', [0, 0, 0, 1, 0, 1, 1, 0, 0, 0]],
 ['1.10.28.30.12.1', [0, 0, 0, 1, 0, 1, 0, 0, 0, 1]],
 ['1.10.28.30.12.1', [0, 0, 0, 0, 1, 1, 1, 0, 0, 0]],
 ['1.10.28.30.12.1', [0, 0, 0, 0, 1, 0, 1, 1, 0, 0]],
 ['1.10.32.36.14.1', [0, 0, 0, 0, 0, 1, 1, 0, 0, 1]],
 ['1.10.32.36.14.1', [0, 0, 0, 0, 0, 1, 0, 0, 1, 1]],
 ['1.10.32.36.14.1', [0, 0, 0, 0, 0, 0, 1, 1, 1, 0]],
 ['1.10.32.36.14.1', [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]],
 ['1.8.24.32.16.1', [1, 1, 1, 0, 1, 0, 0, 0, 0, 0]],
 ['1.8.24.32.16.1', [1, 1, 1, 0, 0, 0, 0, 1, 0, 0]],
 ['1.8.24.32.16.1', [1, 1, 1, 0, 0, 0, 0, 0, 0, 1]],
 ['1.8.24.32.16.1', [1, 1, 0, 1, 1, 0, 0, 0, 0, 0]],
 ['1.8.24.32.16.1', [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]],
 ['1.8.24.32.16.1', [1, 1, 0, 1, 

True

In [13]:
Ptest = Polyhedron(vertices=[(0,2), (-1,0), (1,0), (0,0)])
help(type(Ptest))

Help on class Polyhedra_ZZ_ppl_with_category.element_class in Polyhedra_ZZ_ppl_with_category:

class Polyhedra_ZZ_ppl_with_category.element_class(sage.geometry.polyhedron.backend_ppl.Polyhedron_ZZ_ppl, sage.categories.polyhedra.PolyhedralSets.element_class)
 |  Polyhedra over `\ZZ` with ppl
 |  
 |  INPUT:
 |  
 |  - ``Vrep`` -- a list ``[vertices, rays, lines]`` or ``None``.
 |  
 |  - ``Hrep`` -- a list ``[ieqs, eqns]`` or ``None``.
 |  
 |  EXAMPLES::
 |  
 |      sage: p = Polyhedron(vertices=[(0,0),(1,0),(0,1)], rays=[(1,1)], lines=[],
 |      ....:                backend='ppl', base_ring=ZZ)
 |      sage: TestSuite(p).run(skip='_test_pickling')
 |  
 |  Method resolution order:
 |      Polyhedra_ZZ_ppl_with_category.element_class
 |      sage.geometry.polyhedron.backend_ppl.Polyhedron_ZZ_ppl
 |      sage.geometry.polyhedron.backend_ppl.Polyhedron_ppl
 |      sage.geometry.polyhedron.base_ZZ.Polyhedron_ZZ
 |      sage.geometry.polyhedron.base_QQ.Polyhedron_QQ
 |      sage.geometry

In [ ]:
polymake.congruent(Ptest, Ptest)